# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import linregress
import requests
import time
import datetime
import json

# Import API key
from api_keys import geoapify_key
from citipy import citipy

import warnings
warnings.filterwarnings("ignore")

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,70.92,69.0,100.0,4.00,US,1716905893
1,1,mariental,-24.6333,17.9667,80.85,18.0,40.0,2.30,NaN,1716906709
2,2,mae hong son,19.3003,97.9685,76.66,97.0,100.0,2.01,TH,1716906710
3,3,edinburgh of the seven seas,-37.0676,-12.3116,54.91,59.0,12.0,19.42,SH,1716906711
4,4,tiruchchendur,8.4833,78.1167,85.08,68.0,100.0,10.63,IN,1716906712


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

city_data_df = city_data_df.dropna(how="any")


# Display the map
# YOUR CODE HERE

map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
mask_temp = ((city_data_df['Max Temp'] < 70) & (city_data_df['Max Temp'] > 50))

temp_city_data_df = city_data_df.loc[mask_temp].reset_index()



# Drop any rows with null values
# YOUR CODE HERE
temp_city_data_df.dropna(how="any")

# Display sample data
# YOUR CODE HERE

temp_city_data_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,3,3,edinburgh of the seven seas,-37.0676,-12.3116,54.91,59.0,12.0,19.42,SH,1716906711
1,6,6,adamstown,-25.0660,-130.1015,69.35,79.0,96.0,9.31,PN,1716906714
2,12,12,yellowknife,62.4560,-114.3525,54.41,67.0,100.0,11.50,CA,1716906721
3,13,13,badger,64.8000,-147.5333,53.08,82.0,75.0,4.61,US,1716906291
4,15,15,constantia,44.1833,28.6500,68.70,68.0,5.0,8.61,RO,1716906487


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = temp_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,59.0,
1,adamstown,PN,-25.0660,-130.1015,79.0,
2,yellowknife,CA,62.4560,-114.3525,67.0,
3,badger,US,64.8000,-147.5333,82.0,
4,constantia,RO,44.1833,28.6500,68.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitude = row["Lat"]
 
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    radius = 10000 # YOUR CODE HERE

    categories = "accommodation.hotel"
    limit = 1
    filter = f"circle%3A{longitude}%2C{latitude}%2C{radius}"
    bias =  f"proximity%3A{longitude}%2C{latitude}"
    
    params = {
        "categories": categories,
        "limit": limit,
        "apiKey": geoapify_key,
        "filter": filter,
        "bias": bias
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

# 
    
    # Make and API request using the params dictionaty
    name_address = requests.get(f"{base_url}categories={categories}&limit={limit}&filter={filter}&bias={bias}&apiKey={geoapify_key}")
    
    # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
badger - nearest hotel: Clarion Hotel & Suites
constantia - nearest hotel: Hotel Chérica
susanville - nearest hotel: SureStay Plus Hotel By Best Western Susanville
blackmans bay - nearest hotel: Villa Howden
mohr - nearest hotel: Hotel Heidehof
pacific grove - nearest hotel: Pacific Grove Inn
udachny - nearest hotel: Вилюй
bredasdorp - nearest hotel: Victoria Hotel
mitchell - nearest hotel: Quality Inn
codrington - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
molln - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
mammoth lakes - nearest hotel: Travelodge by Wyndham Mammoth Lakes
santa marta de ortigueira - nearest hotel: Hotel la Perla
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
st. paul - nearest hotel: C

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,59.0,No hotel found
1,adamstown,PN,-25.0660,-130.1015,79.0,No hotel found
2,yellowknife,CA,62.4560,-114.3525,67.0,No hotel found
3,badger,US,64.8000,-147.5333,82.0,Clarion Hotel & Suites
4,constantia,RO,44.1833,28.6500,68.0,Hotel Chérica
...,...,...,...,...,...,...
179,vilhena,BR,-12.7406,-60.1458,77.0,No hotel found
180,lichinga,MZ,-13.3128,35.2406,49.0,Residencial 2+1
181,lhasa,CN,29.6500,91.1000,75.0,Shangri-la Hotel Lhasa
182,sidrolandia,BR,-20.9319,-54.9614,54.0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)